In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime as datatime
from pathlib import Path
import torch
import torch.nn as nn
from PIL import Image
import torch.optim as optim
from torchvision import transforms
from torchvision import models
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from tqdm import tqdm


In [10]:
import zipfile
import os

# 要解压的 zip 文件路径
train_zip = ('./COMP90086_2024_Project_train.zip')
test_zip = ('./COMP90086_2024_Project_test.zip')

# 解压后的目标文件夹（如果不存在则创建）
train_folder = 'COMP90086_2024_Project_train'
os.makedirs(train_folder, exist_ok=True)
test_folder = 'COMP90086_2024_Project_test'
os.makedirs(test_folder, exist_ok=True)

# 打开并解压 zip 文件

with zipfile.ZipFile(train_zip, 'r') as zip_ref:
    zip_ref.extractall(train_folder)

with zipfile.ZipFile(test_zip, 'r') as zip_ref:
    zip_ref.extractall(test_folder)
    
print(f"Files extracted to {train_folder}")
print(f"Files extracted to {test_folder}")

Files extracted to COMP90086_2024_Project_train
Files extracted to COMP90086_2024_Project_test


In [11]:
csv_path = './COMP90086_2024_Project_train/train.csv'
img_dir = './COMP90086_2024_Project_train/train'

## The data set file path is defined here.

In [12]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

'''
In this function block, the funciton is defined to do
1. def__init__() : get the image from the trainning data path
2. def __len__() : get the size of hte dataset
3. def __getitem__() : get the ralated image the correct dataset
'''
class BlockStackDataset(Dataset):
  '''
  Return the image and its label. If not access or not exist, raise error.
  '''
  def __init__(self, data_frame, img_dir, transform = None):
    self.data_frame = data_frame
    self.img_dir = img_dir
    self.transform = transform

    if not os.path.exists(self.img_dir):
      raise ValueError(f"File path {self.img_dir} not exisits!")
    if not os.access(self.img_dir, os.R_OK):
      raise ValueError(f"File path {self.img_dir} is not readable!")
  '''
  Return the size of the dataset
  '''
  def __len__(self):
    return len(self.data_frame)

  '''
  Read the image and according to its id, get the related data in the train.csv.

  Then return the corresponding image , stable_height, instability_type dataset provided.
  '''
  def __getitem__(self, idx):
    img_name = os.path.join(self.img_dir, str(self.data_frame.iloc[idx, 0])) # the first column in the train == image name
    image = Image.open(img_name + ".jpg")
    label = self.data_frame.iloc[idx,-1] # stable_height/instability_type
    # print("image id: ", self.data_frame.iloc[idx,0])
    # print("Whole line: ", self.data_frame.iloc[idx,:].tolist())

    if self.transform:
      image = self.transform(image)

    return image, label



In [13]:
class FineTunedGoogLeNet(nn.Module):
    def __init__(self):
        super(FineTunedGoogLeNet, self).__init__()
        # load the pre-trained model: gogglenet
        self.googlenet = models.googlenet(weights = models.GoogLeNet_Weights.IMAGENET1K_V1)

        num_ftrs = self.googlenet.fc.in_features
        self.googlenet.fc = nn.Identity()

        self.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 6)

        )
    def forward(self, x):
      x = self.googlenet(x)
      x = self.fc(x)
      return x

In [14]:
df = pd.read_csv('./COMP90086_2024_Project_train/train.csv')
df.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
0,54,2,1,3,1,1,2
1,173,1,1,4,1,2,1
2,245,1,1,4,1,2,1
3,465,2,1,5,0,1,5
4,611,2,1,3,1,1,1


## Trainner

In [15]:
'''
The function here helps to create the logs recording the experiment results.
'''
def create_log_dir():
  current_time = datatime.now().strftime("%Y%m%d-%H%M%S")
  # log_dir = f'/content/drive/MyDrive/CV final project/runs/experiment_{current_time}'
  # solution_dir = f'/content/drive/MyDrive/CV final project/trained_models/experiment_{current_time}'

  log_dir = f'./runs/experiment_{current_time}'
  solution_dir = f'./runs/experiment_{current_time}'
  if not os.path.exists(log_dir):
    os.makedirs(log_dir)

  if not os.path.exists(solution_dir):
    os.makedirs(solution_dir)

  return log_dir, solution_dir


In [30]:

# Define models: googlenet

class TunnedBlockStackNet8(nn.Module):
    def __init__(self):
        super(TunnedBlockStackNet8, self).__init__()
        # load the pre-trained model: gogglenet
        self.googlenet = models.googlenet(weights = models.GoogLeNet_Weights.IMAGENET1K_V1)

        num_ftrs = self.googlenet.fc.in_features
        self.googlenet.fc = nn.Identity()

        self.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.50),
            nn.Linear(128, 6)

        )
    def forward(self, x):
      x = self.googlenet(x)
      x = self.fc(x)
      return x

In [40]:
class ClassificationBlockStackTrainer1:
  '''
  The function here helps to initalize the parameters used in the models and pre-process the image
  '''
  def __init__(self, csv_path, img_dir, model, stratify_column='stable_height', test_size=0.2,
                 batch_size=32, num_epochs=10, learning_rate=0.001,random_state=42):
        self.img_dir = img_dir
        self.stratify_column = stratify_column
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.model = model

        # load the train dataset
        self.data_frame = pd.read_csv(csv_path)

        # split data into train and validation dataset
        self.train_data, self.val_data, self.train_ids, self.valid_ids = self.split_dataset()

        hard_df = self.train_data[(self.train_data['type']==2) &
                                  (self.train_data['instability_type']==0)]

        oversample_hard = resample(hard_df,replace=True,n_samples=500)

        hard_df2 = self.train_data[(self.train_data['type']==2) &
                                  (self.train_data['instability_type']==1)]
        oversample_hard2 = resample(hard_df2,replace=True,n_samples=500)

        hard_df3 = self.train_data[(self.train_data['type']==2) &
                                  (self.train_data['instability_type']==2)]

        oversample_hard3 = resample(hard_df3,replace=True,n_samples=200)
        
        conca_df = pd.concat([self.train_data,oversample_hard,oversample_hard2, oversample_hard3])
       # self.train_data = conca_df
        # 打乱顺序
        self.train_data = conca_df.sample(frac=1)

        # pre-processing the images
        self.transform = transforms.Compose([
            # transforms.Resize((512, 512)),

              #ADD MORE TRANSFORM METHODS HERE

            transforms.RandomHorizontalFlip(),## ADDDED
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), ## ADDDED
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),## ADDDED
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


        self.train_loader = self.create_dataloader(self.train_data, self.transform)
        self.val_loader = self.create_dataloader(self.val_data, self.transform, shuffle=False)


        # use the gpu
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # set the loss weight
        '''
        Changed some thing here : to see the results
        1. add the class_weights to each class labels with different weights,
           more lables occurece with lower weight, few label occurence with higher weight;
           without the weights, all the labels will be treated equally.

        2. used to apply L2 regularization (also called weight decay).
        The primary purpose of weight decay is to prevent overfitting by penalizing large weights.
         It adds a penalty to the loss function based on the size of the weights
         helps it generalize better to unseen data.
         Lnew = Lold + weight_decay * sum(weight^2)

        3. add scheduler to Reduces the learning rate after every step_size epochs.
          After every 4 epochs, the learning rate will be multiplied by gamma (0.1 here), reducing it by 90%.
        '''
        class_weights = torch.tensor([100/25 , 100/25, 100/20,  100/15, 100/10,  100/5], device= self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights) # CrossEntropy for multi categorical-label predication
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate , weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size = 15, gamma=0.1)


    def dynamic_oversamp(self):
        if epoch % 3 == 0:
            over_random = 333
        else epoch % 3 == 1:
            over_random == 111
        else:
            over_random = 222
            
        hard_df = self.train_data[(self.train_data['type']==2) &
                                  (self.train_data['instability_type']==0)]

        oversample_hard = resample(hard_df,replace=True,n_samples=500, random)

        hard_df2 = self.train_data[(self.train_data['type']==2) &
                                  (self.train_data['instability_type']==1)]
        oversample_hard2 = resample(hard_df2,replace=True,n_samples=500)

        hard_df3 = self.train_data[(self.train_data['type']==2) &
                                  (self.train_data['instability_type']==2)]

        oversample_hard3 = resample(hard_df3,replace=True,n_samples=200)
        
        conca_df = pd.concat([self.train_data,oversample_hard,oversample_hard2, oversample_hard3])
       # self.train_data = conca_df
        # 打乱顺序
        self.train_data = conca_df.sample(frac=1)
  '''
  The function helps to split the data set into the training and validation dataset according to the
  size pre-determined.
  '''

  def split_dataset(self):
    split = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=42)
    train_ids = []
    valid_ids = []
    for train_idx, val_idx in split.split(self.data_frame, self.data_frame[self.stratify_column]):
      train_data = self.data_frame.iloc[train_idx]
      val_data = self.data_frame.iloc[val_idx]
      train_ids.append(train_idx)
      valid_ids.append(val_idx)
    print(f"Train dataset size: {len(train_data)}",
       f"Validation dataset size: {len(val_data)}",
       f"length of train_ids{(len(train_ids))}",
       f"length of valid_ids{(len(valid_ids))}")
    return train_data, val_data, train_ids, valid_ids


  '''
  The function helps to loda the image
  '''
  def create_dataloader(self, data_frame, transform, shuffle=True):
    dataset = BlockStackDataset(data_frame, self.img_dir, transform=transform) # transform 可以用来数据增强
    return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)


  def generate_classification_report(self, outputs, labels):
    predicted = outputs # 'outputs' is already a numpy array after prediction
    labels = labels
    print(classification_report(labels, predicted, zero_division=0))



  def calculate_confusion_matrix(self, outputs, labels):
    predicted = outputs
    labels = labels
    matrix = confusion_matrix(labels, predicted)
    print(matrix)


  def validate(self):
    self.model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_labels = []  # add all the lables
    all_predictions = []  # add all the prediction

    with torch.no_grad():
        for inputs, labels in self.val_loader:
            inputs, labels = inputs.to(self.device), labels.to(self.device).long()
            labels = labels - 1
            outputs = self.model(inputs)
            predicted = torch.argmax(outputs, 1)

            # collecting all the lables and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # calculate the loss
            loss = self.criterion(outputs, labels)
            val_loss += loss.item()

            # calculate the correct predication
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # calculate the accuracy rate
    val_accuracy = correct_predictions / total_samples
    self.generate_classification_report(np.array(all_predictions), np.array(all_labels))
    self.calculate_confusion_matrix(np.array(all_predictions), np.array(all_labels))
    return val_loss/len(self.val_loader),val_accuracy, all_labels, all_predictions


  '''
  The function here is used as the main training function on the image by using the pre-definned models in
  hte first model class.
  '''

  def train(self):
      _, solution_dir = create_log_dir()

      best_val_accuracy = 0.0
      for epoch in range(self.num_epochs):
          self.model.train()
          running_loss = 0.0
          running_accuracy = 0.0

          # monitor the process
          with tqdm(self.train_loader, unit="batch") as tepoch:
              tepoch.set_description(f"Epoch {epoch + 1}/{self.num_epochs}")

              for inputs, labels in tepoch:
                  inputs, labels = inputs.to(self.device), labels.to(self.device).long()
                  labels = labels - 1

                  #forward propagation
                  self.optimizer.zero_grad()
                  raw_outputs = self.model(inputs)
                  loss = self.criterion(raw_outputs, labels)  # loss calculation

                  loss.backward()  # backward propagation
                  self.optimizer.step()

                  #Loss calculating
                  running_loss += loss.item()
                  _, predicted = torch.max(raw_outputs, 1)
                  accuracy = (predicted == labels).sum().item()/ labels.size(0)
                  running_accuracy += accuracy
                  tepoch.set_postfix(loss=running_loss / len(self.train_loader),
                            accuracy=running_accuracy / len(self.train_loader))



          self.scheduler.step()
          print(self.scheduler.get_last_lr())


          val_loss,val_accuracy, all_labels, all_predictions = self.validate()
          print(f"Epoch {epoch + 1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

          if val_accuracy > best_val_accuracy:
              best_val_accuracy = val_accuracy
              torch.save(self.model.state_dict(), f'{solution_dir}/best_model.pth')
              current_time = datatime.now().strftime("%Y%m%d-%H%M")
              print('Best model saved!', current_time)

      print('Finished Training')
      print(f'Best validation accuracy: {best_val_accuracy:.4f}')



IndentationError: unindent does not match any outer indentation level (<string>, line 68)

In [35]:
class ContinueTrainer:
  '''
  The function here helps to initalize the parameters used in the models and pre-process the image
  '''
  def __init__(self, model_path, model, 
                csv_path, img_dir, 
                stratify_column='stable_height', 
                test_size=0.2,
                batch_size=32, num_epochs=10, learning_rate=0.001 ,random_state=42):
        self.img_dir = img_dir
        self.stratify_column = stratify_column
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate

        self.model = model
        self.model_path = model_path
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model = self.load_model()

        # load the train dataset
        self.data_frame = pd.read_csv(csv_path)

        # split data into train and validation dataset
        self.train_data, self.val_data, self.train_ids, self.valid_ids = self.split_dataset()

       #  hard_df = self.train_data[(self.train_data['type']==2) &
       #                            (self.train_data['instability_type']==0)]

       #  oversample_hard = resample(hard_df,replace=True,n_samples=500)

       #  hard_df2 = self.train_data[(self.train_data['type']==2) &
       #                            (self.train_data['instability_type']==1)]
       #  oversample_hard2 = resample(hard_df2,replace=True,n_samples=200)

       #  hard_df3 = self.train_data[(self.train_data['type']==2) &
       #                            (self.train_data['instability_type']==2)]

       #  oversample_hard3 = resample(hard_df3,replace=True,n_samples=50)
        
       #  conca_df = pd.concat([self.train_data,oversample_hard,oversample_hard2, oversample_hard3])
       # # self.train_data = conca_df
       #  # 打乱顺序
       #  self.train_data = conca_df.sample(frac=1)

        # pre-processing the images
        self.transform = transforms.Compose([
            # transforms.Resize((512, 512)),

              #ADD MORE TRANSFORM METHODS HERE

            transforms.RandomHorizontalFlip(),## ADDDED
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), ## ADDDED
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),## ADDDED
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


        self.train_loader = self.create_dataloader(self.train_data, self.transform)
        self.val_loader = self.create_dataloader(self.val_data, self.transform, shuffle=False)


        # set the loss weight
        '''
        Changed some thing here : to see the results
        1. add the class_weights to each class labels with different weights,
           more lables occurece with lower weight, few label occurence with higher weight;
           without the weights, all the labels will be treated equally.

        2. used to apply L2 regularization (also called weight decay).
        The primary purpose of weight decay is to prevent overfitting by penalizing large weights.
         It adds a penalty to the loss function based on the size of the weights
         helps it generalize better to unseen data.
         Lnew = Lold + weight_decay * sum(weight^2)

        3. add scheduler to Reduces the learning rate after every step_size epochs.
          After every 4 epochs, the learning rate will be multiplied by gamma (0.1 here), reducing it by 90%.
        '''
        class_weights = torch.tensor([100/25 , 100/25, 100/20,  100/15, 100/10,  100/5], device= self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights) # CrossEntropy for multi categorical-label predication
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate , weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size = 10, gamma=0.15)

  def load_model(self):
    self.model.load_state_dict(torch.load(self.model_path, map_location=self.device))
    return self.model

  '''
  The function helps to split the data set into the training and validation dataset according to the
  size pre-determined.
  '''

  def split_dataset(self):
    split = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=42)
    train_ids = []
    valid_ids = []
    for train_idx, val_idx in split.split(self.data_frame, self.data_frame[self.stratify_column]):
      train_data = self.data_frame.iloc[train_idx]
      val_data = self.data_frame.iloc[val_idx]
      train_ids.append(train_idx)
      valid_ids.append(val_idx)
    print(f"Train dataset size: {len(train_data)}",
       f"Validation dataset size: {len(val_data)}",
       f"length of train_ids{(len(train_ids))}",
       f"length of valid_ids{(len(valid_ids))}")
    return train_data, val_data, train_ids, valid_ids


  '''
  The function helps to loda the image
  '''
  def create_dataloader(self, data_frame, transform, shuffle=True):
    dataset = BlockStackDataset(data_frame, self.img_dir, transform=transform) # transform 可以用来数据增强
    return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)


  def generate_classification_report(self, outputs, labels):
    predicted = outputs # 'outputs' is already a numpy array after prediction
    labels = labels
    print(classification_report(labels, predicted, zero_division=0))



  def calculate_confusion_matrix(self, outputs, labels):
    predicted = outputs
    labels = labels
    matrix = confusion_matrix(labels, predicted)
    print(matrix)


  def validate(self):
    self.model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_labels = []  # add all the lables
    all_predictions = []  # add all the prediction

    with torch.no_grad():
        for inputs, labels in self.val_loader:
            inputs, labels = inputs.to(self.device), labels.to(self.device).long()
            labels = labels - 1
            outputs = self.model(inputs)
            predicted = torch.argmax(outputs, 1)

            # collecting all the lables and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # calculate the loss
            loss = self.criterion(outputs, labels)
            val_loss += loss.item()

            # calculate the correct predication
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # calculate the accuracy rate
    val_accuracy = correct_predictions / total_samples
    self.generate_classification_report(np.array(all_predictions), np.array(all_labels))
    self.calculate_confusion_matrix(np.array(all_predictions), np.array(all_labels))
    return val_loss/len(self.val_loader),val_accuracy, all_labels, all_predictions


  '''
  The function here is used as the main training function on the image by using the pre-definned models in
  hte first model class.
  '''

  def train(self):
      _, solution_dir = create_log_dir()

      best_val_accuracy = 0.0
      for epoch in range(self.num_epochs):
          self.model.train()
          running_loss = 0.0
          running_accuracy = 0.0

          # monitor the process
          with tqdm(self.train_loader, unit="batch") as tepoch:
              tepoch.set_description(f"Epoch {epoch + 1}/{self.num_epochs}")

              for inputs, labels in tepoch:
                  inputs, labels = inputs.to(self.device), labels.to(self.device).long()
                  labels = labels - 1

                  #forward propagation
                  self.optimizer.zero_grad()
                  raw_outputs = self.model(inputs)
                  loss = self.criterion(raw_outputs, labels)  # loss calculation

                  loss.backward()  # backward propagation
                  self.optimizer.step()

                  #Loss calculating
                  running_loss += loss.item()
                  _, predicted = torch.max(raw_outputs, 1)
                  accuracy = (predicted == labels).sum().item()/ labels.size(0)
                  running_accuracy += accuracy
                  tepoch.set_postfix(loss=running_loss / len(self.train_loader),
                            accuracy=running_accuracy / len(self.train_loader))



          self.scheduler.step()
          print(self.scheduler.get_last_lr())


          val_loss,val_accuracy, all_labels, all_predictions = self.validate()
          print(f"Epoch {epoch + 1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

          if val_accuracy > best_val_accuracy:
              best_val_accuracy = val_accuracy
              torch.save(self.model.state_dict(), f'{solution_dir}/best_model.pth')
              current_time = datatime.now().strftime("%Y%m%d-%H%M")
              print('Best model saved!', current_time)

      print('Finished Training')
      print(f'Best validation accuracy: {best_val_accuracy:.4f}')



# Model 3
* with higher batch size: 128

In [38]:
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer3 = ClassificationBlockStackTrainer1(
        csv_path = csv_path, 
        img_dir=img_dir, 
        model=model,
        test_size=0.2, 
        num_epochs=64,
        batch_size=128,
        )
    trainer3.train()

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/64: 100%|██████████| 58/58 [01:10<00:00,  1.22s/batch, accuracy=0.25, loss=1.6]     


[0.001]
              precision    recall  f1-score   support

           0       0.14      0.02      0.03       384
           1       0.38      0.47      0.42       384
           2       0.39      0.38      0.39       307
           3       0.23      0.71      0.34       230
           4       0.00      0.00      0.00       154
           5       0.00      0.00      0.00        77

    accuracy                           0.30      1536
   macro avg       0.19      0.26      0.20      1536
weighted avg       0.24      0.30      0.24      1536

[[  6 197  63 118   0   0]
 [  6 179  74 125   0   0]
 [  7  52 117 131   0   0]
 [  5  17  44 164   0   0]
 [  7  16   0 131   0   0]
 [ 11   5   2  59   0   0]]
Epoch 1/64, Validation Loss: 1.4900, Validation Accuracy: 0.3034
Best model saved! 20241014-2309


Epoch 2/64: 100%|██████████| 58/58 [01:00<00:00,  1.05s/batch, accuracy=0.342, loss=1.44]   


[0.001]
              precision    recall  f1-score   support

           0       0.60      0.35      0.45       384
           1       0.49      0.35      0.41       384
           2       0.45      0.35      0.40       307
           3       0.28      0.65      0.40       230
           4       0.20      0.36      0.26       154
           5       0.00      0.00      0.00        77

    accuracy                           0.38      1536
   macro avg       0.34      0.34      0.32      1536
weighted avg       0.43      0.38      0.38      1536

[[136  73  47  84  44   0]
 [ 69 134  49  87  45   0]
 [ 12  58 108  92  37   0]
 [  3   7  34 150  36   0]
 [  5   3   0  91  55   0]
 [  1   0   0  23  53   0]]
Epoch 2/64, Validation Loss: 1.3897, Validation Accuracy: 0.3796
Best model saved! 20241014-2310


...

[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.71      0.65      0.68       384
           1       0.72      0.67      0.70       384
           2       0.68      0.68      0.68       307
           3       0.58      0.63      0.60       230
           4       0.53      0.58      0.55       154
           5       0.36      0.49      0.42        77

    accuracy                           0.64      1536
   macro avg       0.60      0.62      0.60      1536
weighted avg       0.65      0.64      0.65      1536

[[250  42  33  36  15   8]
 [ 40 257  33  31  11  12]
 [ 22  26 209  23  16  11]
 [ 18  11  20 144  22  15]
 [ 15   9   7  12  89  22]
 [  6  10   7   2  14  38]]
Epoch 62/64, Validation Loss: 0.9664, Validation Accuracy: 0.6426


Epoch 63/64: 100%|██████████| 58/58 [01:11<00:00,  1.23s/batch, accuracy=0.768, loss=0.641] 


[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.75      0.68      0.71       384
           1       0.75      0.68      0.72       384
           2       0.66      0.70      0.68       307
           3       0.62      0.66      0.64       230
           4       0.55      0.58      0.57       154
           5       0.37      0.51      0.43        77

    accuracy                           0.66      1536
   macro avg       0.62      0.64      0.62      1536
weighted avg       0.67      0.66      0.67      1536

[[262  36  29  33  10  14]
 [ 38 261  34  27  13  11]
 [ 18  25 216  20  11  17]
 [ 16   7  23 152  22  10]
 [ 15   9  14  13  89  14]
 [  2   8  10   2  16  39]]
Epoch 63/64, Validation Loss: 0.9456, Validation Accuracy: 0.6634


Epoch 64/64: 100%|██████████| 58/58 [01:13<00:00,  1.26s/batch, accuracy=0.762, loss=0.657] 


[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.72      0.65      0.68       384
           1       0.74      0.66      0.70       384
           2       0.66      0.69      0.68       307
           3       0.60      0.66      0.63       230
           4       0.52      0.58      0.55       154
           5       0.29      0.38      0.33        77

    accuracy                           0.64      1536
   macro avg       0.59      0.60      0.59      1536
weighted avg       0.65      0.64      0.65      1536

[[250  36  35  34  16  13]
 [ 39 254  34  27  14  16]
 [ 20  26 213  23  13  12]
 [ 16  10  19 151  21  13]
 [ 13   8  10  15  90  18]
 [  9   8  10   2  19  29]]
Epoch 64/64, Validation Loss: 1.0035, Validation Accuracy: 0.6426
Finished Training
Best validation accuracy: 0.6654


# Model2
* with higher batch size: 64

In [37]:
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer2 = ClassificationBlockStackTrainer1(
        csv_path = csv_path, 
        img_dir=img_dir, 
        model=model,
        test_size=0.2, 
        num_epochs=64,
        batch_size=64
        )
    trainer2.train()

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/64:   0%|          | 0/115 [00:00<?, ?batch/s]/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Epoch 1/64: 100%|██████████| 115/115 [01:08<00:00,  1.67batch/s, accuracy=0.265, loss=1.61]  


[0.001]
              precision    recall  f1-score   support

           0       0.39      0.58      0.47       384
           1       0.00      0.00      0.00       384
           2       0.33      0.22      0.27       307
           3       0.20      0.67      0.31       230
           4       0.00      0.00      0.00       154
           5       0.00      0.00      0.00        77

    accuracy                           0.29      1536
   macro avg       0.15      0.25      0.17      1536
weighted avg       0.20      0.29      0.22      1536

[[221   0  37 126   0   0]
 [189   0  61 134   0   0]
 [104   0  69 134   0   0]
 [ 44   0  31 155   0   0]
 [  2   0  10 142   0   0]
 [  1   0   0  76   0   0]]
Epoch 1/64, Validation Loss: 1.5434, Validation Accuracy: 0.2897
Best model saved! 20241014-2142


Epoch 2/64: 100%|██████████| 115/115 [01:11<00:00,  1.62batch/s, accuracy=0.337, loss=1.47]  


[0.001]
              precision    recall  f1-score   support

           0       0.29      0.49      0.36       384
           1       0.41      0.41      0.41       384
           2       0.37      0.47      0.42       307
           3       0.41      0.07      0.11       230
           4       0.33      0.16      0.22       154
           5       0.00      0.00      0.00        77

    accuracy                           0.35      1536
   macro avg       0.30      0.27      0.25      1536
weighted avg       0.34      0.35      0.32      1536

[[188 119  61   6  10   0]
 [117 157  99   4   7   0]
 [ 86  60 145   8   8   0]
 [ 80  43  69  15  22   1]
 [109   6  11   3  25   0]
 [ 69   1   2   1   4   0]]
Epoch 2/64, Validation Loss: 1.4333, Validation Accuracy: 0.3451
Best model saved! 20241014-2144


...

[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.76      0.61      0.68       384
           1       0.75      0.66      0.70       384
           2       0.65      0.68      0.67       307
           3       0.55      0.63      0.59       230
           4       0.47      0.56      0.51       154
           5       0.29      0.44      0.35        77

    accuracy                           0.63      1536
   macro avg       0.58      0.60      0.58      1536
weighted avg       0.65      0.63      0.63      1536

[[235  38  35  39  18  19]
 [ 35 253  36  29  14  17]
 [  9  28 209  30  17  14]
 [ 14   5  25 145  32   9]
 [ 10   6   8  17  87  26]
 [  8   6   8   3  18  34]]
Epoch 62/64, Validation Loss: 0.9103, Validation Accuracy: 0.6270


Epoch 63/64: 100%|██████████| 115/115 [01:00<00:00,  1.90batch/s, accuracy=0.699, loss=0.792] 


[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.77      0.64      0.70       384
           1       0.74      0.66      0.70       384
           2       0.64      0.69      0.66       307
           3       0.59      0.62      0.60       230
           4       0.47      0.60      0.53       154
           5       0.37      0.49      0.42        77

    accuracy                           0.64      1536
   macro avg       0.60      0.62      0.60      1536
weighted avg       0.66      0.64      0.65      1536

[[246  37  39  28  25   9]
 [ 28 254  39  30  17  16]
 [ 16  30 213  23  13  12]
 [ 13   8  23 142  33  11]
 [ 11   6  11  16  92  18]
 [  4   6  10   3  16  38]]
Epoch 63/64, Validation Loss: 0.9270, Validation Accuracy: 0.6413


Epoch 64/64: 100%|██████████| 115/115 [01:05<00:00,  1.76batch/s, accuracy=0.7, loss=0.793]    


[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.75      0.65      0.70       384
           1       0.72      0.67      0.70       384
           2       0.65      0.66      0.65       307
           3       0.62      0.60      0.61       230
           4       0.48      0.59      0.53       154
           5       0.28      0.44      0.34        77

    accuracy                           0.63      1536
   macro avg       0.58      0.60      0.59      1536
weighted avg       0.65      0.63      0.64      1536

[[251  38  33  25  21  16]
 [ 37 257  38  19  19  14]
 [ 13  28 202  30  18  16]
 [ 15  13  23 138  25  16]
 [ 11  10   8   8  91  26]
 [  7   9   9   2  16  34]]
Epoch 64/64, Validation Loss: 0.9101, Validation Accuracy: 0.6335
Finished Training
Best validation accuracy: 0.6523


In [36]:
# test
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_path = csv_path, 
        img_dir=img_dir, 
        model=model,
        test_size=0.2, 
        num_epochs=64,
        batch_size=32
        )
    trainer.train()

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/64: 100%|██████████| 230/230 [01:05<00:00,  3.53batch/s, accuracy=0.256, loss=1.61]   


[0.001]
              precision    recall  f1-score   support

           0       0.52      0.40      0.45       384
           1       0.00      0.00      0.00       384
           2       0.26      0.46      0.33       307
           3       0.19      0.59      0.29       230
           4       0.00      0.00      0.00       154
           5       0.00      0.00      0.00        77

    accuracy                           0.28      1536
   macro avg       0.16      0.24      0.18      1536
weighted avg       0.21      0.28      0.22      1536

[[152   0 125 107   0   0]
 [ 99   0 157 128   0   0]
 [ 35   0 140 132   0   0]
 [  6   0  88 136   0   0]
 [  1   0  17 136   0   0]
 [  0   0   2  75   0   0]]
Epoch 1/64, Validation Loss: 1.5360, Validation Accuracy: 0.2786
Best model saved! 20241014-2020


Epoch 2/64: 100%|██████████| 230/230 [01:01<00:00,  3.71batch/s, accuracy=0.308, loss=1.53]   


[0.001]
              precision    recall  f1-score   support

           0       0.87      0.23      0.37       384
           1       0.45      0.44      0.44       384
           2       0.34      0.25      0.29       307
           3       0.21      0.76      0.33       230
           4       0.00      0.00      0.00       154
           5       0.00      0.00      0.00        77

    accuracy                           0.33      1536
   macro avg       0.31      0.28      0.24      1536
weighted avg       0.43      0.33      0.31      1536

[[ 90 119  45 130   0   0]
 [ 10 169  49 156   0   0]
 [  0  78  76 153   0   0]
 [  1   7  48 174   0   0]
 [  2   3   3 146   0   0]
 [  0   1   0  76   0   0]]
Epoch 2/64, Validation Loss: 1.4856, Validation Accuracy: 0.3314
Best model saved! 20241014-2021


...

[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.70      0.62      0.66       384
           1       0.73      0.61      0.66       384
           2       0.63      0.66      0.65       307
           3       0.58      0.57      0.57       230
           4       0.45      0.54      0.49       154
           5       0.27      0.49      0.35        77

    accuracy                           0.60      1536
   macro avg       0.56      0.58      0.56      1536
weighted avg       0.63      0.60      0.61      1536

[[240  39  43  35  14  13]
 [ 39 233  38  26  23  25]
 [ 16  24 203  25  17  22]
 [ 13  11  22 130  33  21]
 [ 23   9   8   8  83  23]
 [ 11   5   8   0  15  38]]
Epoch 62/64, Validation Loss: 0.9816, Validation Accuracy: 0.6035


Epoch 63/64: 100%|██████████| 230/230 [01:10<00:00,  3.28batch/s, accuracy=0.606, loss=0.983]  


[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.69      0.63      0.66       384
           1       0.73      0.61      0.66       384
           2       0.64      0.64      0.64       307
           3       0.55      0.56      0.56       230
           4       0.44      0.56      0.49       154
           5       0.27      0.44      0.33        77

    accuracy                           0.60      1536
   macro avg       0.55      0.57      0.56      1536
weighted avg       0.62      0.60      0.61      1536

[[242  34  40  35  17  16]
 [ 39 233  39  29  23  21]
 [ 20  27 195  31  16  18]
 [ 16   9  22 129  39  15]
 [ 20  10   5   9  86  24]
 [ 16   5   5   1  16  34]]
Epoch 63/64, Validation Loss: 0.9881, Validation Accuracy: 0.5983


Epoch 64/64: 100%|██████████| 230/230 [01:06<00:00,  3.48batch/s, accuracy=0.611, loss=0.987]  


[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.72      0.64      0.68       384
           1       0.72      0.60      0.65       384
           2       0.62      0.60      0.61       307
           3       0.57      0.58      0.58       230
           4       0.43      0.55      0.48       154
           5       0.28      0.56      0.38        77

    accuracy                           0.60      1536
   macro avg       0.56      0.59      0.56      1536
weighted avg       0.63      0.60      0.61      1536

[[245  36  36  33  14  20]
 [ 41 230  42  28  17  26]
 [ 19  32 183  26  27  20]
 [ 11   7  19 134  39  20]
 [ 17   8   8  14  84  23]
 [  8   7   6   0  13  43]]
Epoch 64/64, Validation Loss: 0.9907, Validation Accuracy: 0.5983
Finished Training
Best validation accuracy: 0.6133


### Model with oversampling hard examples in instability type 1
* add 500 hard examples in instability type 1
* need increase epoches

In [16]:
# test
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=16,
        batch_size=32
        )
    trainer.train()
        

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/16: 100%|██████████| 208/208 [06:05<00:00,  1.76s/batch, accuracy=0.284, loss=1.57]    


[0.001]
              precision    recall  f1-score   support

           0       0.61      0.33      0.43       384
           1       0.32      0.75      0.44       384
           2       0.25      0.26      0.25       307
           3       0.18      0.07      0.11       230
           4       0.29      0.01      0.02       154
           5       0.00      0.00      0.00        77

    accuracy                           0.33      1536
   macro avg       0.27      0.24      0.21      1536
weighted avg       0.34      0.33      0.29      1536

[[128 195  48  13   0   0]
 [ 40 287  42  13   2   0]
 [ 11 203  79  13   1   0]
 [ 14 113  85  17   1   0]
 [ 12  72  46  22   2   0]
 [  4  40  17  15   1   0]]
Epoch 1/16, Validation Loss: 1.8216, Validation Accuracy: 0.3340
Best model saved! 20241014-1742


Epoch 2/16: 100%|██████████| 208/208 [06:03<00:00,  1.75s/batch, accuracy=0.371, loss=1.46]   


[0.001]
              precision    recall  f1-score   support

           0       0.53      0.51      0.52       384
           1       0.47      0.41      0.44       384
           2       0.32      0.46      0.38       307
           3       0.19      0.26      0.22       230
           4       0.17      0.07      0.10       154
           5       0.21      0.04      0.07        77

    accuracy                           0.37      1536
   macro avg       0.32      0.29      0.29      1536
weighted avg       0.37      0.37      0.36      1536

[[196  59  69  44  13   3]
 [ 82 159  81  54   8   0]
 [ 37  83 141  40   5   1]
 [ 22  27 108  59  10   4]
 [ 24   8  31  77  11   3]
 [ 10   5  14  29  16   3]]
Epoch 2/16, Validation Loss: 1.4574, Validation Accuracy: 0.3704
Best model saved! 20241014-1749


...

[0.001]
              precision    recall  f1-score   support

           0       0.57      0.69      0.62       384
           1       0.66      0.67      0.66       384
           2       0.61      0.54      0.57       307
           3       0.54      0.48      0.51       230
           4       0.39      0.47      0.42       154
           5       0.28      0.06      0.11        77

    accuracy                           0.57      1536
   macro avg       0.51      0.48      0.48      1536
weighted avg       0.56      0.57      0.56      1536

[[264  58  34  18   8   2]
 [ 45 256  37  20  24   2]
 [ 46  33 166  38  21   3]
 [ 49  20  12 111  38   0]
 [ 37  11   9  19  72   6]
 [ 22  11  15   0  24   5]]
Epoch 14/16, Validation Loss: 1.1130, Validation Accuracy: 0.5690


Epoch 15/16: 100%|██████████| 208/208 [06:11<00:00,  1.79s/batch, accuracy=0.569, loss=1.08]  


[0.0001]
              precision    recall  f1-score   support

           0       0.75      0.66      0.70       384
           1       0.69      0.60      0.64       384
           2       0.61      0.62      0.61       307
           3       0.65      0.44      0.52       230
           4       0.34      0.66      0.45       154
           5       0.30      0.34      0.32        77

    accuracy                           0.59      1536
   macro avg       0.55      0.55      0.54      1536
weighted avg       0.62      0.59      0.60      1536

[[252  40  34  17  31  10]
 [ 38 231  41  17  39  18]
 [ 17  29 191  17  40  13]
 [ 17  22  26 101  58   6]
 [ 12  10  13   4 102  13]
 [  2   5  10   0  34  26]]
Epoch 15/16, Validation Loss: 1.0438, Validation Accuracy: 0.5879
Best model saved! 20241014-1920


Epoch 16/16: 100%|██████████| 208/208 [06:09<00:00,  1.78s/batch, accuracy=0.607, loss=0.991] 


[0.0001]
              precision    recall  f1-score   support

           0       0.83      0.59      0.69       384
           1       0.72      0.65      0.68       384
           2       0.59      0.68      0.63       307
           3       0.57      0.57      0.57       230
           4       0.41      0.58      0.48       154
           5       0.28      0.44      0.34        77

    accuracy                           0.61      1536
   macro avg       0.57      0.58      0.57      1536
weighted avg       0.65      0.61      0.62      1536

[[225  45  42  33  27  12]
 [ 15 249  51  26  19  24]
 [ 10  24 209  27  24  13]
 [  7  14  23 130  41  15]
 [ 10   7  15   9  89  24]
 [  5   5  13   2  18  34]]
Epoch 16/16, Validation Loss: 0.9651, Validation Accuracy: 0.6094
Best model saved! 20241014-1927
Finished Training
Best validation accuracy: 0.6094


In [18]:
valid_ds = trainer.val_data
_,val_accuracy, all_labels, all_predictions = trainer.validate()
valid_ds['predicted'] = all_predictions
valid_ds.head()

              precision    recall  f1-score   support

           0       0.78      0.56      0.65       384
           1       0.71      0.66      0.68       384
           2       0.60      0.69      0.64       307
           3       0.55      0.53      0.54       230
           4       0.43      0.58      0.50       154
           5       0.23      0.36      0.28        77

    accuracy                           0.60      1536
   macro avg       0.55      0.56      0.55      1536
weighted avg       0.63      0.60      0.61      1536

[[214  50  44  35  24  17]
 [ 18 253  48  24  22  19]
 [ 13  24 212  28  16  14]
 [ 11  14  24 121  44  16]
 [ 11   4  13   9  90  27]
 [  9  11  14   2  13  28]]


/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_95107/284521822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_ds['predicted'] = all_predictions


,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,predicted
7334,956915,2,2,4,2,1,1,0
3955,516709,1,2,5,1,1,3,1
619,77447,1,2,5,1,1,3,2
1594,212770,2,2,4,2,1,1,0
5645,745098,2,2,6,1,1,4,4


In [20]:
valid_ds['predicted'] = valid_ds['predicted'] + 1
valid_ds['pred_type'] = valid_ds['predicted'] == valid_ds['stable_height']

valid_ds.pred_type.sum()/len(valid_ds)

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_95107/911565123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_ds['predicted'] = valid_ds['predicted'] + 1
/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_95107/911565123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_ds['pred_type'] = valid_ds['predicted'] == valid_ds['stable_height']


0.59765625

In [21]:
valid_ds.groupby(['instability_type', 'type']).pred_type.sum()/valid_ds.groupby(['instability_type', 'type']).pred_type.count()

instability_type  type
0                 1       0.796407
                  2       0.608911
1                 1       0.500000
                  2       0.299742
2                 1       0.898058
                  2       0.902174
Name: pred_type, dtype: float64

In [9]:
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=16,
        batch_size=32
        )
    trainer.train()
        

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/16: 100%|██████████| 192/192 [06:25<00:00,  2.01s/batch, accuracy=0.621, loss=0.814]  


[0.001]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       369
           1       0.63      0.99      0.77       777
           2       0.98      0.78      0.87       390

    accuracy                           0.70      1536
   macro avg       0.54      0.59      0.55      1536
weighted avg       0.57      0.70      0.61      1536

[[  0 368   1]
 [  0 773   4]
 [  0  87 303]]
Epoch 1/16, Validation Loss: 0.6589, Validation Accuracy: 0.7005
Best model saved!


Epoch 2/16: 100%|██████████| 192/192 [06:23<00:00,  2.00s/batch, accuracy=0.689, loss=0.664]  


[0.001]
              precision    recall  f1-score   support

           0       0.67      0.05      0.09       369
           1       0.65      0.97      0.78       777
           2       0.93      0.83      0.87       390

    accuracy                           0.71      1536
   macro avg       0.75      0.62      0.58      1536
weighted avg       0.72      0.71      0.64      1536

[[ 18 340  11]
 [  8 754  15]
 [  1  66 323]]
Epoch 2/16, Validation Loss: 0.6186, Validation Accuracy: 0.7129
Best model saved!


...

[0.001]
              precision    recall  f1-score   support

           0       0.54      0.47      0.50       369
           1       0.75      0.82      0.78       777
           2       0.97      0.90      0.93       390

    accuracy                           0.76      1536
   macro avg       0.75      0.73      0.74      1536
weighted avg       0.75      0.76      0.75      1536

[[172 192   5]
 [134 637   6]
 [ 13  26 351]]
Epoch 14/16, Validation Loss: 0.5306, Validation Accuracy: 0.7552


Epoch 15/16: 100%|██████████| 192/192 [06:20<00:00,  1.98s/batch, accuracy=0.775, loss=0.517]  


[0.0001]
              precision    recall  f1-score   support

           0       0.63      0.37      0.47       369
           1       0.73      0.90      0.81       777
           2       0.97      0.90      0.93       390

    accuracy                           0.77      1536
   macro avg       0.78      0.72      0.74      1536
weighted avg       0.77      0.77      0.76      1536

[[137 225   7]
 [ 75 697   5]
 [  6  32 352]]
Epoch 15/16, Validation Loss: 0.5107, Validation Accuracy: 0.7721


Epoch 16/16: 100%|██████████| 192/192 [06:20<00:00,  1.98s/batch, accuracy=0.786, loss=0.48]   


[0.0001]
              precision    recall  f1-score   support

           0       0.59      0.41      0.48       369
           1       0.74      0.86      0.79       777
           2       0.96      0.93      0.94       390

    accuracy                           0.77      1536
   macro avg       0.76      0.73      0.74      1536
weighted avg       0.76      0.77      0.76      1536

[[151 212   6]
 [101 666  10]
 [  5  22 363]]
Epoch 16/16, Validation Loss: 0.4951, Validation Accuracy: 0.7682
Finished Training
Best validation accuracy: 0.7754


In [10]:
train_ids = trainer.train_ids
valid_ids = trainer.valid_ids

In [12]:
val_loss,val_accuracy, all_labels, all_predictions = trainer.validate()

              precision    recall  f1-score   support

           0       0.62      0.41      0.49       369
           1       0.75      0.88      0.81       777
           2       0.97      0.93      0.95       390

    accuracy                           0.78      1536
   macro avg       0.78      0.74      0.75      1536
weighted avg       0.77      0.78      0.77      1536

[[152 211   6]
 [ 88 683   6]
 [  7  19 364]]


In [13]:
comparison = map(lambda all_labels, all_predictions: all_labels == all_predictions, all_labels, all_predictions)
sum(comparison)/len(all_labels)

0.7805989583333334

In [14]:
val_data = trainer.val_data
val_data.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
7334,956915,2,2,4,2,1,1
3955,516709,1,2,5,1,1,3
619,77447,1,2,5,1,1,3
1594,212770,2,2,4,2,1,1
5645,745098,2,2,6,1,1,4


In [15]:
val_data['predicted'] = all_predictions
val_data.head()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_37333/1695572929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['predicted'] = all_predictions


,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,predicted
7334,956915,2,2,4,2,1,1,2
3955,516709,1,2,5,1,1,3,1
619,77447,1,2,5,1,1,3,1
1594,212770,2,2,4,2,1,1,2
5645,745098,2,2,6,1,1,4,1


In [16]:
val_data['pred_type'] = val_data['predicted'] == val_data['instability_type']
val_data.head()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_37333/3086623943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['pred_type'] = val_data['predicted'] == val_data['instability_type']


,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,predicted,pred_type
7334,956915,2,2,4,2,1,1,2,True
3955,516709,1,2,5,1,1,3,1,True
619,77447,1,2,5,1,1,3,1,True
1594,212770,2,2,4,2,1,1,2,True
5645,745098,2,2,6,1,1,4,1,True


In [18]:
len(val_data[val_data['predicted']== val_data['instability_type']])/len(valid_ids[0])

0.7805989583333334

In [88]:
val_data['pred_type'].sum()/len(val_data)

0.64453125

In [91]:
val_data.groupby('instability_type').pred_type.count()

instability_type
0    369
1    777
2    390
Name: pred_type, dtype: int64

In [22]:
val_data.groupby('shapeset').pred_type.sum()/val_data.groupby('shapeset').pred_type.count()

shapeset
1    0.812734
2    0.763473
Name: pred_type, dtype: float64

In [21]:
val_data.groupby('type').pred_type.sum()/val_data.groupby('type').pred_type.count()

type
1    0.821756
2    0.739974
Name: pred_type, dtype: float64

In [20]:
val_data.groupby('instability_type').pred_type.sum()/val_data.groupby('instability_type').pred_type.count()

instability_type
0    0.411924
1    0.879022
2    0.933333
Name: pred_type, dtype: float64

In [23]:
val_data.groupby('cam_angle').pred_type.sum()/val_data.groupby('cam_angle').pred_type.count()

cam_angle
1    0.802426
2    0.714660
Name: pred_type, dtype: float64

In [3]:
import pandas as pd
df = pd.read_csv('./COMP90086_2024_Project_train/train.csv')
df.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
0,54,2,1,3,1,1,2
1,173,1,1,4,1,2,1
2,245,1,1,4,1,2,1
3,465,2,1,5,0,1,5
4,611,2,1,3,1,1,1


In [6]:
df.groupby('instability_type').count()

,id,shapeset,type,total_height,cam_angle,stable_height
instability_type,,,,,,
0,1920,1920,1920,1920,1920,1920
1,3840,3840,3840,3840,3840,3840
2,1920,1920,1920,1920,1920,1920


In [10]:
best_model = torch.load('./runs/experiment_20241012-010838/best_model.pth')

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_67928/1284981419.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load('./runs/experiment_20

In [15]:
best_model = torch.load('./runs/experiment_20241012-010838/best_model.pth',map_location=torch.device('cpu'))
best_model.eval()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_67928/1886415100.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load('./runs/experiment_20

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

# Load Model

Test Model

Net8, epoch = 8, no batch_size defined

In [13]:
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        # csv_file = '/content/drive/MyDrive/CV final project/train data/train.csv', ##
        # img_dir='/content/drive/MyDrive/CV final project/train data/train', ##
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=8,
        #batch_size=32
        )
    trainer.train()

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /Users/macbookpro/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:03<00:00, 14.0MB/s]


Train dataset size: 6144 Validation dataset size: 1536


Epoch 1/8: 100%|██████████| 192/192 [06:38<00:00,  2.07s/batch, accuracy=0.274, loss=1.58]   


[0.001]
              precision    recall  f1-score   support

           0       0.49      0.42      0.45       384
           1       0.38      0.45      0.41       384
           2       0.30      0.62      0.41       307
           3       0.25      0.06      0.09       230
           4       0.26      0.12      0.16       154
           5       0.00      0.00      0.00        77

    accuracy                           0.36      1536
   macro avg       0.28      0.28      0.25      1536
weighted avg       0.34      0.36      0.33      1536

[[161 134  78   4   7   0]
 [ 70 172 132   4   6   0]
 [ 26  71 191  12   7   0]
 [ 28  47 124  13  18   0]
 [ 32  12  76  16  18   0]
 [ 13  11  35   4  14   0]]
Epoch 1/8, Validation Loss: 1.5266, Validation Accuracy: 0.3613
Best model saved!


Epoch 2/8: 100%|██████████| 192/192 [06:44<00:00,  2.11s/batch, accuracy=0.363, loss=1.45]   


...

[0.001]
              precision    recall  f1-score   support

           0       0.77      0.38      0.51       384
           1       0.69      0.51      0.58       384
           2       0.53      0.53      0.53       307
           3       0.42      0.47      0.44       230
           4       0.25      0.69      0.37       154
           5       0.12      0.12      0.12        77

    accuracy                           0.47      1536
   macro avg       0.46      0.45      0.43      1536
weighted avg       0.56      0.47      0.49      1536

[[147  52  53  45  80   7]
 [ 23 194  65  32  46  24]
 [  6  17 162  56  51  15]
 [  9   5  19 109  84   4]
 [  6   6   6  17 106  13]
 [  1   8   0   1  58   9]]
Epoch 6/8, Validation Loss: 1.3388, Validation Accuracy: 0.4733


Epoch 7/8: 100%|██████████| 192/192 [06:46<00:00,  2.12s/batch, accuracy=0.49, loss=1.22]    


[0.001]
              precision    recall  f1-score   support

           0       0.44      0.70      0.54       384
           1       0.51      0.58      0.54       384
           2       0.48      0.23      0.31       307
           3       0.42      0.33      0.37       230
           4       0.40      0.38      0.39       154
           5       0.31      0.05      0.09        77

    accuracy                           0.46      1536
   macro avg       0.43      0.38      0.37      1536
weighted avg       0.45      0.46      0.43      1536

[[268  78  14  18   6   0]
 [103 222  22  20  15   2]
 [ 72 101  71  47  14   2]
 [ 66  20  33  76  35   0]
 [ 63   7   5  15  59   5]
 [ 38   9   4   3  19   4]]
Epoch 7/8, Validation Loss: 1.3688, Validation Accuracy: 0.4557


Epoch 8/8: 100%|██████████| 192/192 [06:54<00:00,  2.16s/batch, accuracy=0.515, loss=1.2]    


[0.001]
              precision    recall  f1-score   support

           0       0.64      0.58      0.61       384
           1       0.60      0.60      0.60       384
           2       0.48      0.57      0.52       307
           3       0.47      0.50      0.49       230
           4       0.36      0.36      0.36       154
           5       0.26      0.13      0.17        77

    accuracy                           0.53      1536
   macro avg       0.47      0.46      0.46      1536
weighted avg       0.52      0.53      0.52      1536

[[223  80  40  29  10   2]
 [ 47 231  71  18  14   3]
 [ 21  49 174  41  20   2]
 [ 24   8  48 116  28   6]
 [ 23   7  17  36  55  16]
 [ 12  10  13   7  25  10]]
Epoch 8/8, Validation Loss: 1.1596, Validation Accuracy: 0.5267
Best model saved!
Finished Training
Best validation accuracy: 0.5267
